<a href="https://colab.research.google.com/github/abelalanoca/P.MAGNO/blob/main/TP1/Balance_y_ER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from google.colab import files, drive
import pandas as pd

# 1️⃣ Subir archivo desde tu PC
uploaded = files.upload()

# 2️⃣ Montar Google Drive (si quieres guardar/leer desde ahí)
drive.mount("/content/drive")

# 3️⃣ Leer hoja 1 del archivo subido
nombre_archivo = list(uploaded.keys())[0]
df_raw = pd.read_excel(nombre_archivo, sheet_name=0)

# --------------------------------------------------------
# 🔹 Función: Normalizar nombres de bancos
# --------------------------------------------------------
def reemplazar_bancos_especificos(df):
    """
    Normaliza nombres específicos de bancos en el DataFrame.
    Retorna un nuevo DataFrame limpio.
    """
    df_1 = df.copy()

    reemplazos = {
        "COMPARTAMOS BANCO": "Compartamos Banco*",
        "BANCO DE COMERCIO": "BANCOM"
    }

    df_1 = df_1.applymap(
        lambda x: reemplazos.get(str(x).strip().upper(), x) if isinstance(x, str) else x
    )
    return df_1

df_1 = reemplazar_bancos_especificos(df_raw)

# --------------------------------------------------------
# 🔹 Función: Extraer listas de cuentas y bancos
# --------------------------------------------------------
def extraer_listas_balance(df):
    primera_col = df.iloc[:, 0].astype(str).str.strip().str.upper()

    try:
        idx_activo = primera_col[primera_col == "ACTIVO"].index[0]
        idx_total_activo = primera_col[primera_col == "TOTAL ACTIVO"].index[0]
    except IndexError:
        raise ValueError("No se encontraron filas 'ACTIVO' o 'TOTAL ACTIVO' en el archivo.")

    cuentas_del_activo = (
        df.iloc[idx_activo:idx_total_activo + 1, 0]
        .dropna().astype(str).str.strip().tolist()
    )
    cuentas_del_activo = [c for c in cuentas_del_activo if c.upper() != "ACTIVO"]

    try:
        idx_pasivo = primera_col[primera_col == "PASIVO"].index[0]
        idx_otras_conting = primera_col[primera_col == "OTRAS CUENTAS CONTINGENTES"].index[0]
    except IndexError:
        raise ValueError("No se encontraron filas 'PASIVO' u 'OTRAS CUENTAS CONTINGENTES'.")

    lista_pasivos = (
        df.iloc[idx_pasivo:idx_otras_conting + 1, 0]
        .dropna().astype(str).str.strip().tolist()
    )
    lista_pasivos = [p for p in lista_pasivos if p.upper() != "PASIVO"]

    bancos_activo = (
        df.iloc[idx_activo, 1:]
        .dropna().astype(str).str.strip().tolist()
    )
    bancos_activo = [b for b in bancos_activo if b.upper() != "ACTIVO"]

    return cuentas_del_activo, lista_pasivos, bancos_activo

cuentas_del_activo, lista_pasivos, bancos_activo = extraer_listas_balance(df_1)

# --------------------------------------------------------
# 🔹 Función: Extraer fecha
# --------------------------------------------------------
def extraer_fecha(df):
    idx_balance = df[df.astype(str).apply(
        lambda row: row.str.contains("Balance General por Empresa Bancaria", case=False, na=False)
    ).any(axis=1)].index[0]

    fecha_cruda = df.iloc[idx_balance + 1].dropna().astype(str).tolist()[0]

    fecha_dt = pd.to_datetime(fecha_cruda, errors="coerce")
    return [fecha_dt.strftime("%b-%d") if pd.notnull(fecha_dt) else fecha_cruda]

Fecha = extraer_fecha(df_1)

# --------------------------------------------------------
# 🔹 Función: Crear estructura vacía df_2
# --------------------------------------------------------
def crear_df_2(Fecha, bancos_activo, cuentas_del_activo, lista_pasivos):
    def limpiar_lista(lista):
        return [x for x in lista if pd.notnull(x) and str(x).strip() != ""]

    cuentas_del_activo = limpiar_lista(cuentas_del_activo)
    lista_pasivos = limpiar_lista(lista_pasivos)
    bancos_activo = limpiar_lista(bancos_activo)

    columnas_cuentas = cuentas_del_activo + lista_pasivos
    df_2 = pd.DataFrame(columns=["MES", "ENTIDAD", "MONEDA"] + columnas_cuentas)

    entidades, monedas, meses = [], [], []
    for banco in bancos_activo:
        for moneda in ["MN", "ME", "TOTAL"]:
            entidades.append(banco)
            monedas.append(moneda)
            meses.append(Fecha[0])

    df_2["MES"] = meses
    df_2["ENTIDAD"] = entidades
    df_2["MONEDA"] = monedas

    return df_2

df_2 = crear_df_2(Fecha, bancos_activo, cuentas_del_activo, lista_pasivos)

# --------------------------------------------------------
# 🔹 Función: Llenar ACTIVO o PASIVO en df_2
# --------------------------------------------------------
def llenar_df_2_matriz(df_1, df_2, bancos, cuentas, tipo="ACTIVO"):
    """
    Llena df_2 con valores de ACTIVO o PASIVO según el tipo.
    """
    df_limpio = df_1.copy().applymap(lambda x: str(x).strip() if pd.notnull(x) else x)

    # índice de sección
    idx_tipo = df_limpio[df_limpio.iloc[:, 0].str.upper() == tipo].index[0]

    # filas de cuentas
    filas_cuentas = {}
    for cuenta in cuentas:
        match = df_limpio.index[df_limpio.iloc[:, 0].str.upper() == cuenta.upper()].tolist()
        if match:
            filas_cuentas[cuenta] = match[0]

    # columnas de bancos
    columnas_bancos = {}
    for banco in bancos:
        match = df_limpio.columns[df_limpio.iloc[idx_tipo, :].astype(str) == banco].tolist()
        if match:
            columnas_bancos[banco] = df_limpio.columns.get_loc(match[0])

    # llenar valores
    for banco, col_idx in columnas_bancos.items():
        for cuenta, fila_idx in filas_cuentas.items():
            valores = df_1.iloc[fila_idx, col_idx:col_idx+3].tolist()
            for moneda, valor in zip(["MN", "ME", "TOTAL"], valores):
                df_2.loc[(df_2["ENTIDAD"] == banco) & (df_2["MONEDA"] == moneda), cuenta] = valor

    return df_2

# --------------------------------------------------------
# 🔹 Pipeline completo
# --------------------------------------------------------
df_2 = llenar_df_2_matriz(df_1, df_2, bancos_activo, cuentas_del_activo, tipo="ACTIVO")
df_2 = llenar_df_2_matriz(df_1, df_2, bancos_activo, lista_pasivos, tipo="PASIVO")

# limpiar encabezados
df_2.columns = df_2.columns.map(lambda x: ''.join(ch for ch in str(x) if ch.isalpha() or ch.isspace()))

Saving B-2201-jn2025 (1).XLS to B-2201-jn2025 (1) (1).XLS
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/tmp/ipython-input-2420172600.py:29: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_1 = df_1.applymap(
/tmp/ipython-input-2420172600.py:127: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_limpio = df_1.copy().applymap(lambda x: str(x).strip() if pd.notnull(x) else x)
/tmp/ipython-input-2420172600.py:127: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_limpio = df_1.copy().applymap(lambda x: str(x).strip() if pd.notnull(x) else x)


In [17]:
import pandas as pd
import itertools
import re
from google.colab import files

# --------------------------------------------------------
# 🔹 Extraer lista de cuentas del ER (solo primera columna) - ajustado
# --------------------------------------------------------
primera_col_hoja2 = df_raw_hoja2.iloc[:, 0].astype(str).str.strip()

try:
    idx_ingresos = primera_col_hoja2[primera_col_hoja2.str.contains("INGRESOS FINANCIEROS", case=False, na=False)].index[0]
    idx_resultado = primera_col_hoja2[primera_col_hoja2.str.contains("RESULTADO NETO DEL EJERCICIO", case=False, na=False)].index[0]
except IndexError:
    raise ValueError("No se encontraron 'INGRESOS FINANCIEROS' o 'RESULTADO NETO DEL EJERCICIO'.")

# Ajuste clave: incluir la fila final con 'RESULTADO NETO DEL EJERCICIO'
Cuentas_del_ER = df_raw_hoja2.iloc[idx_ingresos:idx_resultado + 1, 0].dropna().astype(str).str.strip().tolist()

# --------------------------------------------------------
# 🔹 Extraer lista de bancos del ER (ajustado con desplazamiento)
# --------------------------------------------------------
fila_bancos = idx_ingresos - 3  # los bancos están 3 filas arriba
Bancos_del_ER = (
    df_raw_hoja2.iloc[fila_bancos, 1:]   # tomamos desde columna 1 en adelante
    .dropna()
    .astype(str)
    .str.strip()
    .tolist()
)

# --------------------------------------------------------
# 🔹 Crear df_3
# --------------------------------------------------------
columnas_ER = ["MES", "ENTIDAD", "MONEDA"] + Cuentas_del_ER
df_3 = pd.DataFrame(columns=columnas_ER)

# Repetir cada banco 3 veces para MN, ME, TOTAL
entidades = list(itertools.chain.from_iterable([[banco]*3 for banco in Bancos_del_ER]))
monedas = ["MN", "ME", "TOTAL"] * len(Bancos_del_ER)
meses = [Fecha[0]] * len(entidades)

df_3["ENTIDAD"] = entidades
df_3["MONEDA"] = monedas
df_3["MES"] = meses

# --------------------------------------------------------
# 🔹 Función de normalización
# --------------------------------------------------------
def normalizar(texto):
    texto = str(texto).upper()
    texto = re.sub(r"[ÁÀÄÂ]", "A", texto)
    texto = re.sub(r"[ÉÈËÊ]", "E", texto)
    texto = re.sub(r"[ÍÌÏÎ]", "I", texto)
    texto = re.sub(r"[ÓÒÖÔ]", "O", texto)
    texto = re.sub(r"[ÚÙÜÛ]", "U", texto)
    texto = re.sub(r"[\(\)]", "", texto)
    texto = re.sub(r"\s+", " ", texto)
    return texto.strip()

# Limpiar df
df_limpio = df_raw_hoja2.copy().astype(str).apply(lambda col: col.str.strip())
df_limpio.iloc[:, 0] = df_limpio.iloc[:, 0].map(normalizar)

# --------------------------------------------------------
# 🔹 Llenado general de df_3 para todos los bancos
# --------------------------------------------------------
for banco in Bancos_del_ER:
    banco_norm = normalizar(banco)

    # 1️⃣ Buscar fila del banco (coincidencia parcial)
    mask_fila_banco = df_limpio.apply(lambda row: row.astype(str).map(normalizar).str.contains(banco_norm).any(), axis=1)
    if not mask_fila_banco.any():
        print(f"No se encontró el banco: {banco}")
        continue
    fila_banco_idx = mask_fila_banco[mask_fila_banco].index[0]

    # 2️⃣ Recorrer todas las cuentas
    for cuenta in Cuentas_del_ER:
        cuenta_norm = normalizar(cuenta)
        mask_cuenta = df_limpio.iloc[:, 0].str.contains(cuenta_norm)

        if not mask_cuenta.any():
            print(f"No se encontró la cuenta (después de normalizar): {cuenta}")
            continue

        fila_cuenta_idx = mask_cuenta[mask_cuenta].index[0]

        # 3️⃣ Columna del banco
        mask_col = df_limpio.iloc[fila_banco_idx, :].map(normalizar).str.contains(banco_norm)
        if not mask_col.any():
            print(f"No se encontró la columna del banco '{banco}' en la fila del banco")
            continue

        col_banco_idx = df_limpio.columns.get_loc(mask_col[mask_col].index[0])

        # 4️⃣ Extraer valores MN, ME, TOTAL
        valores = df_raw_hoja2.iloc[fila_cuenta_idx, col_banco_idx:col_banco_idx+3].tolist()

        # 5️⃣ Llenar df_3
        for moneda, valor in zip(["MN", "ME", "TOTAL"], valores):
            df_3.loc[(df_3["ENTIDAD"] == banco) & (df_3["MONEDA"] == moneda), cuenta] = valor

# --------------------------------------------------------
# 🔹 Visualizar df_3 completo y exportar
# --------------------------------------------------------


In [18]:
!pip install xlsxwriter
import pandas as pd
from google.colab import files

archivo_salida = "Balance_y_ER.xlsx"

with pd.ExcelWriter(archivo_salida, engine='xlsxwriter') as writer:
    df_2.to_excel(writer, sheet_name="Balance_General", index=False)
    df_3.to_excel(writer, sheet_name="Estado_de_Resultados", index=False)

files.download(archivo_salida)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>